In [1]:
import re
from PyPDF2 import PdfFileReader, PdfFileWriter
import PyPDF2
import pandas as pd
from bertopic import BERTopic 
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora;
import pickle
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer



In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Predefined Functions

In [3]:
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)
    #removing \n
    s = re.sub(r'\n',r' ', s)
    #removing §
    s = re.sub(r'§',r' ', s)
    #remove numbers
    s = re.sub(r'([0-9]q|[0-9]|-\*[0-9]|(\w)[0-9])',r'',s)
    #remove -'s
    s = re.sub(r'(-|–\*|. –)',r'',s)
     #remove -'s
    #s = re.sub(r'-',r'',s)
    
    #remove roman numbers
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    s = re.sub(pattern, '', s)
    

    return s.strip()

In [4]:
def read_pdf_data(file):
    data = []
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj,strict=False)
    number_of_pages = len(pdfReader.pages)
    for line in range(0,number_of_pages):
        #line_data = pdfReader.getPage(line).extractText().replace(" -\n","")
        line_data = f_base(pdfReader.getPage(line).extractText().replace(" -\n",""))
        data.append(line_data)
    return data

In [5]:
data_dict = {}
for year in range(18, 23):
    for week in range(1,49):
        print(f"Reading data for year: {year}-{week}")
        if week > 9: 
            data_dict[str(year)+"-"+str(week)] = read_pdf_data(f"..//data//irb{year}-{week}.pdf")
        else:
            data_dict[str(year)+"-"+"0"+str(week)] = read_pdf_data(f"..//data//irb{year}-0{week}.pdf")

Reading data for year: 18-1
Reading data for year: 18-2
Reading data for year: 18-3
Reading data for year: 18-4
Reading data for year: 18-5
Reading data for year: 18-6
Reading data for year: 18-7
Reading data for year: 18-8
Reading data for year: 18-9
Reading data for year: 18-10
Reading data for year: 18-11
Reading data for year: 18-12
Reading data for year: 18-13
Reading data for year: 18-14
Reading data for year: 18-15
Reading data for year: 18-16
Reading data for year: 18-17
Reading data for year: 18-18
Reading data for year: 18-19
Reading data for year: 18-20
Reading data for year: 18-21
Reading data for year: 18-22
Reading data for year: 18-23
Reading data for year: 18-24
Reading data for year: 18-25
Reading data for year: 18-26
Reading data for year: 18-27
Reading data for year: 18-28
Reading data for year: 18-29
Reading data for year: 18-30
Reading data for year: 18-31
Reading data for year: 18-32
Reading data for year: 18-33
Reading data for year: 18-34
Reading data for year: 

In [6]:
import warnings
warnings.filterwarnings("ignore")

## Reading Docs

In [7]:
#data_list = [data_dict[str(year)+"-"+str(week)] for year in range(18,23) for week in range(1, 49)]
data_list = []
for year in range(18, 23):
    for week in range(1,49):
        #print(f"Adding data for year: {year}-{week}")
        if week > 9: 
            data_list.append(data_dict[str(year)+"-"+str(week)])
            print(f"Adding data for year: {year}-{week}")
        else:
            data_list.append(data_dict[str(year)+"-"+"0"+str(week)])
            print(f"Adding data for year: {year}-0{week}")

Adding data for year: 18-01
Adding data for year: 18-02
Adding data for year: 18-03
Adding data for year: 18-04
Adding data for year: 18-05
Adding data for year: 18-06
Adding data for year: 18-07
Adding data for year: 18-08
Adding data for year: 18-09
Adding data for year: 18-10
Adding data for year: 18-11
Adding data for year: 18-12
Adding data for year: 18-13
Adding data for year: 18-14
Adding data for year: 18-15
Adding data for year: 18-16
Adding data for year: 18-17
Adding data for year: 18-18
Adding data for year: 18-19
Adding data for year: 18-20
Adding data for year: 18-21
Adding data for year: 18-22
Adding data for year: 18-23
Adding data for year: 18-24
Adding data for year: 18-25
Adding data for year: 18-26
Adding data for year: 18-27
Adding data for year: 18-28
Adding data for year: 18-29
Adding data for year: 18-30
Adding data for year: 18-31
Adding data for year: 18-32
Adding data for year: 18-33
Adding data for year: 18-34
Adding data for year: 18-35
Adding data for year

In [8]:
flat_list = []
for sublist in data_list:
    for item in sublist:
        flat_list.append(item)
        
len(flat_list)

11638

In [9]:
key_list =[]
key_dict = {"18":"2018","19":"2019","20":"2020","21":"2021","22":"2022"}
for key in data_dict.keys():
    for val in data_dict[key]:
        key_list.append(key_dict[key.split('-')[0]])
len(key_list)

11638

In [10]:
set(key_list)

{'2018', '2019', '2020', '2021', '2022'}

## Identifying Stopwords

In [11]:
# from keybert import KeyBERT
# # Extract keywords
# kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(flat_list)

In [12]:
#keywords[0:20]

## Data Clensing

In [13]:
custom_stop_words = []

# Folder Path
path = "stopwords"
  
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        for word in f.readlines():
            custom_stop_words.append(word.lower().replace('\n',''))

for (root, dirs, file) in os.walk(path): 
    for f in file:
        if '.txt' in f:
            print(f)
            read_text_file(path+"/"+f)


In [14]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)

## Sentence Transformers

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
model_basic = BERTopic(verbose=True, embedding_model=sentence_model, min_topic_size= 10)
topics_basic, probs_basic = model_basic.fit_transform(flat_list)

Batches:   0%|          | 0/364 [00:00<?, ?it/s]

In [ ]:
freq_basic = model_basic.get_topic_info()
print("Number of topics: {}".format( len(freq_basic)))
freq_basic.head()

In [ ]:
model_basic.get_representative_docs(0)

In [ ]:
df_basic = pd.DataFrame({"topic":topics_basic,"docs": flat_list})
df_basic

In [ ]:
model_basic.visualize_topics()

In [ ]:
model_basic.visualize_barchart(top_n_topics=20)

## TF-IDF Vectorizer

In [ ]:
docs = flat_list

vectorizer = TfidfVectorizer(min_df=25)
embeddings = vectorizer.fit_transform(docs)

vectorizer_model = CountVectorizer(stop_words=my_stop_words)
topic_model = BERTopic(vectorizer_model=vectorizer_model, n_gram_range=(2, 3))
#topic_model = BERTopic(n_gram_range=(2, 3))


topics, probs = topic_model.fit_transform(docs, embeddings)

In [ ]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

In [ ]:
topic_model.get_topic(1)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=20)

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', low_memory=False)
topic_model_umap = BERTopic(umap_model=umap_model,vectorizer_model=vectorizer_model, n_gram_range=(2, 3)).fit(flat_list)

In [ ]:
topic_model_umap.visualize_barchart(top_n_topics=10)

In [ ]:
topic_model_umap.visualize_documents(docs, embeddings=embeddings)


In [ ]:
reduced_embeddings = UMAP(n_neighbors=40, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model_umap.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

## c-TF-IDF

In [ ]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model_ctf_idf = BERTopic(ctfidf_model=ctfidf_model,vectorizer_model=vectorizer_model).fit(flat_list)

In [ ]:
freq_m2 = topic_model_ctf_idf.get_topic_info()
print("Number of topics: {}".format( len(freq_m2)))
freq_m2.head()
#.visualize_barchart(top_n_topics=100)

In [ ]:
topic_model_ctf_idf.visualize_barchart(top_n_topics=30)

In [ ]:
topics_over_time = topic_model_ctf_idf.topics_over_time(flat_list, key_list, nr_bins=20)

topic_model_ctf_idf.visualize_topics_over_time(topics_over_time, topics=[4, 5, 7, 12, 13, 24, 25])

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model_hdb = BERTopic(hdbscan_model=hdbscan_model,vectorizer_model=vectorizer_model).fit(flat_list)


In [ ]:
topic_model_hdb.visualize_barchart(top_n_topics=20)

## Finding Perplexity With LDA


In [ ]:
len(flat_list)

In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = flat_list[:5000]
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(custom_stop_words)
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)